In [4]:
pip install fastapi

You should consider upgrading via the 'c:\Users\DELA FARAHITA ZEIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install "uvicorn[standard]"

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELA FARAHITA ZEIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install python-multipart

  Using cached python_multipart-0.0.19-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELA FARAHITA ZEIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
pip install opencv-python-headless

You should consider upgrading via the 'c:\Users\DELA FARAHITA ZEIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


In [9]:
pip install opencv-contrib-python


  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\DELA FARAHITA ZEIN\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\DELA FARAHITA ZEIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [13]:
pip uninstall opencv-python opencv-contrib-python


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install opencv-contrib-python==4.5.5.64


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\DELA FARAHITA ZEIN\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\DELA FARAHITA ZEIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [16]:
pip install dlib face_recognition

  Running setup.py clean for dlib
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566199 sha256=edecfe02c2d3a4e4ecb39577394b382369cb1cff3839b09a09d5301615b8b85e
  Stored in directory: c:\users\dela farahita zein\appdata\local\pip\cache\wheels\7a\eb\cf\e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
Failed to build dlib
    Running setup.py install for dlib: started
    Running setup.py install for dlib: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'c:\Users\DELA FARAHITA ZEIN\AppData\Local\Programs\Python\Python310\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\DELA FARAHITA ZEIN\\AppData\\Local\\Temp\\pip-install-6qa9mths\\dlib_28a3b8efc403422a96c70b3f20caa3db\\setup.py'"'"'; __file__='"'"'C:\\Users\\DELA FARAHITA ZEIN\\AppData\\Local\\Temp\\pip-install-6qa9mths\\dlib_28a3b8efc403422a96c70b3f20caa3db\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\DELA FARAHITA ZEIN\AppData\Local\Temp\pip-wheel-t1ib9wq5'
       cwd: C:\Users\DELA FARAHITA ZEIN\AppData\Local\Temp\pip-install-6qa9mths\dlib_28a3b8efc403422a96c70b3f20caa3db\
  Complete output (41 lines):
  running bdist_wheel
  runn

In [18]:
import os
import cv2
import numpy as np
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import tempfile
import shutil

app = FastAPI()

# Load Haar Cascade Classifier
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

# Load pre-trained face recognition model
model = cv2.face.LBPHFaceRecognizer_create()
model.read("lbph_model.yml")

# Load labels
# labels = ["Dela", "Kinata", "Mulki", "Pascalis"]  
labels = ["Colin_Powell", "Donald_Rumsfeld", "George_W_Bush", "Gerhard_Schroeder", "Tony_Blair"]  

def detect_face(img):
    """
    Detect and crop face from an image
    
    Args:
        img (numpy.ndarray): Input image
    
    Returns:
        numpy.ndarray or None: Processed face image
    """
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # If no face found, return None
    if len(faces) == 0:
        return None
    
    # Take the first detected face
    x, y, w, h = faces[0]
    
    # Crop and resize the face
    face_img = gray[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (100, 100))
    
    return face_img

@app.post("/recognize/")
async def recognize_face(file: UploadFile = File(...)):
    try:
        # Create a temporary file to save the uploaded image
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
            shutil.copyfileobj(file.file, temp_file)
            temp_file_path = temp_file.name
        
        # Read the image
        img = cv2.imread(temp_file_path)
        
        # Remove the temporary file
        os.unlink(temp_file_path)
        
        # Detect and preprocess face
        face_img = detect_face(img)
        
        # If no face detected
        if face_img is None:
            return JSONResponse(content={
                "error": "No face detected in the image"
            }, status_code=400)
        
        # Predict using the model
        idx, confidence = model.predict(face_img)
        
        # Return prediction results
        return JSONResponse(content={
            "predicted_name": labels[idx],
            "confidence": float(confidence)
        })
    
    except Exception as e:
        return JSONResponse(content={
            "error": str(e)
        }, status_code=500)

# Optional: Health check endpoint
@app.get("/")
async def health_check():
    return {"status": "Face Recognition API is running"}

AttributeError: module 'cv2' has no attribute 'face'

In [20]:
import cv2
import numpy as np
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import tempfile
import shutil

app = FastAPI()

# Load Haar Cascade Classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Daftar label yang dikenal
labels = ["Colin_Powell", "Donald_Rumsfeld", "George_W_Bush", "Gerhard_Schroeder", "Tony_Blair"]

def detect_face(img):
    """
    Deteksi dan crop wajah dari gambar
    """
    # Konversi ke grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Deteksi wajah
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Jika tidak ada wajah, kembalikan None
    if len(faces) == 0:
        return None
    
    # Ambil wajah pertama
    x, y, w, h = faces[0]
    
    # Crop dan resize wajah
    face_img = gray[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (100, 100))
    
    return face_img

@app.post("/recognize/")
async def recognize_face(file: UploadFile = File(...)):
    try:
        # Buat file sementara
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
            shutil.copyfileobj(file.file, temp_file)
            temp_file_path = temp_file.name
        
        # Baca gambar
        img = cv2.imread(temp_file_path)
        
        # Hapus file sementara
        os.unlink(temp_file_path)
        
        # Deteksi wajah
        face_img = detect_face(img)
        
        # Jika tidak ada wajah terdeteksi
        if face_img is None:
            return JSONResponse(content={
                "error": "Tidak ada wajah terdeteksi"
            }, status_code=400)
        
        # Simulasi prediksi (ganti dengan model sebenarnya)
        return JSONResponse(content={
            "predicted_name": labels[0],  # Contoh: selalu kembalikan label pertama
            "confidence": 70.0
        })
    
    except Exception as e:
        return JSONResponse(content={
            "error": str(e)
        }, status_code=500)

@app.get("/")
async def health_check():
    return {"status": "Face Recognition API is running"}

In [21]:
from fastapi import FastAPI, UploadFile, File
import cv2
import numpy as np
import os
import tempfile
import shutil

app = FastAPI()

# Load Haar Cascade Classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Daftar label yang dikenal
labels = ["Colin_Powell", "Donald_Rumsfeld", "George_W_Bush", "Gerhard_Schroeder", "Tony_Blair"]

def detect_face(img):
    """
    Deteksi dan crop wajah dari gambar
    """
    # Konversi ke grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Deteksi wajah
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Jika tidak ada wajah, kembalikan None
    if len(faces) == 0:
        return None
    
    # Ambil wajah pertama
    x, y, w, h = faces[0]
    
    # Crop dan resize wajah
    face_img = gray[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (100, 100))
    
    return face_img

@app.post("/recognize/")
async def recognize_face(file: UploadFile = File(...)):
    try:
        # Buat file sementara
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
            shutil.copyfileobj(file.file, temp_file)
            temp_file_path = temp_file.name
        
        # Baca gambar
        img = cv2.imread(temp_file_path)
        
        # Hapus file sementara
        os.unlink(temp_file_path)
        
        # Deteksi wajah
        face_img = detect_face(img)
        
        # Jika tidak ada wajah terdeteksi
        if face_img is None:
            return {
                "error": "Tidak ada wajah terdeteksi"
            }
        
        # Simulasi prediksi (ganti dengan model sebenarnya)
        return {
            "predicted_name": labels[0],  # Contoh: selalu kembalikan label pertama
            "confidence": 70.0
        }
    
    except Exception as e:
        return {
            "error": str(e)
        }

@app.get("/")
async def health_check():
    return {"status": "Face Recognition API is running"}